In [2]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/My Drive/APS360 Group/Data preprocessing/Images/Images_with_CC.zip" -d "/content/drive/My Drive/APS360 Group/Data preprocessing/Images/"

import os
import shutil

# Define the base directory
base_dir = "/content/drive/My Drive/APS360 Group/Data preprocessing/Images/"

#imgae titled with the identifier (zeina 123), then a csv with that identifyer, and the caption

# Define the folders for each sentiment
sentiment_folders = {
    'negative': ['worried teenager', 'sad child', 'scary house'], #10,000 per class
    'neutral': ['placeholder_neutral_1', 'placeholder_neutral_2'], # Replace placeholders with actual names
    'positive': ['placeholder_positive_1', 'placeholder_positive_2'] # Replace placeholders with actual names
}

# Function to move files from source to destination
def move_files(source_folder, dest_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        dest_path = os.path.join(dest_folder, filename)
        if os.path.isfile(source_path):
            shutil.move(source_path, dest_path)
        elif os.path.isdir(source_path):
            if not os.path.exists(dest_path):
                os.makedirs(dest_path)
            move_files(source_path, dest_path)
            os.rmdir(source_path) # Remove the now-empty source folder

# Iterate over the sentiment categories
for sentiment, folders in sentiment_folders.items():
    sentiment_folder_path = os.path.join(base_dir, sentiment)
    if not os.path.exists(sentiment_folder_path):
        os.makedirs(sentiment_folder_path) # Create sentiment folder if it doesn't exist

    # Move the images from each specified folder to the sentiment folder
    for folder_name in folders:
        source_folder_path = os.path.join(base_dir, folder_name)
        if os.path.exists(source_folder_path):
            move_files(source_folder_path, sentiment_folder_path)
            os.rmdir(source_folder_path) # Remove the original folder once it's empty


KeyboardInterrupt: 

In [ ]:
import os
import shutil

# Define the base directory
base_dir = "/content/drive/My Drive/APS360 Group/Data preprocessing/Images/"

# Define the folders for each sentiment
sentiment_folders = {
    'negative': ['worried teenager', 'sad child', 'scary house'],
    'neutral': ['placeholder_neutral_1', 'placeholder_neutral_2'], # Replace placeholders with actual names
    'positive': ['placeholder_positive_1', 'placeholder_positive_2'] # Replace placeholders with actual names
}

# Function to move files from source to destination
def move_files(source_folder, dest_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        dest_path = os.path.join(dest_folder, filename)
        if os.path.isfile(source_path):
            shutil.move(source_path, dest_path)
        elif os.path.isdir(source_path):
            new_dest_folder = os.path.join(dest_folder, filename)
            if not os.path.exists(new_dest_folder):
                os.makedirs(new_dest_folder)
            move_files(source_path, new_dest_folder)
            # Original folder is not deleted

# Iterate over the sentiment categories
for sentiment, folders in sentiment_folders.items():
    sentiment_folder_path = os.path.join(base_dir, sentiment)
    if not os.path.exists(sentiment_folder_path):
        os.makedirs(sentiment_folder_path) # Create sentiment folder if it doesn't exist

    # Move the images from each specified folder to the sentiment folder
    for folder_name in folders:
        source_folder_path = os.path.join(base_dir, folder_name)
        if os.path.exists(source_folder_path):
            move_files(source_folder_path, sentiment_folder_path)
            # Original folder is not deleted


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import shutil
import re
import json
import pandas as pd

# Define the base directory and CSV file path
base_dir = "/content/drive/My Drive/APS360 Group/Data preprocessing/Images/"
csv_file_path = os.path.join(base_dir, "image_descriptions.csv")

# Initialize a DataFrame to store image descriptions
df_descriptions = pd.DataFrame(columns=["id", "caption", "sentiment"])

# Define sentiment values
sentiment_values = {'negative': -1, 'neutral': 0, 'positive': 1}

# Image renaming setup
image_counter = 1

# Function to clean text by removing unwanted characters
def clean_text(text):
    return re.sub(r'[^A-Za-z ]', '', text)

# Function to extract and concatenate content from json.text files
def extract_content(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
        content_parts = [data.get('title', {}).get('_content', ''), data.get('description', {}).get('_content', '')]
        cleaned_parts = [clean_text(part) for part in content_parts]
        return ' '.join(cleaned_parts).strip()

# Function to move and rename files, and update the DataFrame
def process_files(source_folder, dest_folder, sentiment_label):
    global image_counter
    for filename in os.listdir(source_folder):
        if filename.endswith(".jpg"):
            # Rename and move the image
            new_filename = f"Zeina{image_counter}.jpeg"
            shutil.move(os.path.join(source_folder, filename), os.path.join(dest_folder, new_filename))

            # Find and process the corresponding .json.text file
            json_filename = filename.rsplit('.', 1)[0] + ".json.text"
            json_path = os.path.join(source_folder, json_filename)
            if os.path.exists(json_path):
                caption = extract_content(json_path)
                df_descriptions.loc[image_counter] = [new_filename, caption, sentiment_values[sentiment_label]]

            image_counter += 1

# Define the folders for each sentiment
sentiment_folders = {
    'negative': ['disgusting_food', 'nasty_bugs'],
    'neutral': ['dry_food', 'quiet_river'],
    'positive': ['cheerful_smile', 'cute_kids']
}

# Create and process sentiment folders
for sentiment, folders in sentiment_folders.items():
    sentiment_folder_path = os.path.join(base_dir, sentiment)
    if not os.path.exists(sentiment_folder_path):
        os.makedirs(sentiment_folder_path)

    # Process each specified source folder
    for folder_name in folders:
        source_folder_path = os.path.join(base_dir, folder_name)
        if os.path.exists(source_folder_path):
            process_files(source_folder_path, sentiment_folder_path, sentiment)

# Save the DataFrame to a CSV file
df_descriptions.to_csv(csv_file_path, index=False)
